In [ ]:
import mitsuba as mi
mi.set_variant("cuda_ad_rgb")
import importlib
import application
import vapl_config 
import vapl_grid
import vapl_utils
import integrator
importlib.reload(vapl_config)
importlib.reload(application)
importlib.reload(vapl_grid)
importlib.reload(vapl_utils)
importlib.reload(integrator)

app = application.Application(vapl_config.config)
app.train()